<a href="https://colab.research.google.com/github/jbloewencolon/Psychedelic-Trip-Generator/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import re


In [8]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/processed.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77088 entries, 0 to 77087
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             76974 non-null  object
 1   drug              76969 non-null  object
 2   dosage            74319 non-null  object
 3   delivery          74718 non-null  object
 4   weight            77088 non-null  int64 
 5   year              77088 non-null  int64 
 6   gender            76967 non-null  object
 7   report            76959 non-null  object
 8   processed_report  76959 non-null  object
 9   mixed             77088 non-null  int64 
 10  drug_category     77088 non-null  object
dtypes: int64(3), object(8)
memory usage: 6.5+ MB


In [12]:
# Drop rows with missing 'drug_category' or 'processed_report'
df = df.dropna(subset=['drug_category', 'processed_report'])

In [13]:
df.head()

,title,drug,dosage,delivery,weight,year,gender,report,processed_report,mixed,drug_category
0,Ode to Joy,mdma,1.5 tablets,oral,185,2000,male,My friend had some experience with X and had t...,friend experi x told one day said come across ...,0,mdma
1,Make Sure the Music's Not Too Complex,cannabis,unknown,smoked,152,1999,not specified,This was the first experience that either my f...,first experi either friend salvia housem check...,0,cannabis
2,After Hours,mdma,160 mg,oral,150,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,mdma
3,After Hours,mdma,100 mg,oral,150,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,mdma
4,After Hours,mdma,50 mg,insufflated,150,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...,1,mdma


In [14]:
# Instantiate the TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the processed reports
X = vectorizer.fit_transform(df['processed_report']).toarray()

# Let's say you're going to predict the 'drug' column, convert it to a format suitable for the model.
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(df['drug_category'])

In [ ]:
# Number of topics you want to find
n_topics = 10

# Create and fit the LDA model
lda = LDA(n_components=n_topics)
lda.fit(X)

# Function to print the top words for each topic
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Print the top 10 words per topic
print_top_words(lda, vectorizer.get_feature_names(), 10)

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Logistic Regression model
lr = LogisticRegression()

# Train the model
lr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr.predict(X_test)

# Print a classification report
print(classification_report(y_test, y_pred))


In [ ]:
# Use a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict the test set results
y_pred = clf.predict(X_test)

# Evaluate the performance of the model
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

In [ ]:
# Instantiate the XGBClassifier
xgb = XGBClassifier(use_label_encoder=False)

# Fit the model to the training data
xgb.fit(X_train, y_train, eval_metric='mlogloss')

# Predict the labels of the test set
y_pred = xgb.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# Define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# Create a GridSearchCV object
grid_search = GridSearchCV(lr, param_grid, cv=5)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(grid_search.best_params_)
print(grid_search.best_score_)
